### 0. Download the dataset 

The data can be downloaded using the kagglehub package, and the downloaded files will then be moved to the current working directory. Make sure to install kagglehub if it's not already installed.
This dataset consists of images from two categories: Cats and Dogs. It is well-suited for a binary classification task, with approximately 25,000 images.


In [3]:
# pip install kagglehub
import os
import shutil
import kagglehub 

def download_kaggle(dataset_name):
    # Download the dataset
    path = kagglehub.dataset_download(dataset_name)

    print("Path to dataset files:", path)

    # Get the current working directory
    current_dir = os.getcwd()

    # Iterate through all files in the `path` directory
    for filename in os.listdir(path):
        source_file = os.path.join(path, filename)  # Original file path
        destination_file = os.path.join(current_dir, filename)  # Destination file path

        # If the destination exists, check whether it's a file or a folder
        if os.path.exists(destination_file):
            if os.path.isfile(destination_file):  # If it's a file, remove it
                os.remove(destination_file)
            elif os.path.isdir(destination_file):  # If it's a folder, remove it recursively
                shutil.rmtree(destination_file)

        # Move file to the current directory
        shutil.move(source_file, destination_file)
        print(f"Moved: {filename} -> {destination_file}")

    # Ensure `path` is empty before deleting the folder
    if not os.listdir(path):  # Ensure the directory is empty
        os.rmdir(path)
        print(f"Deleted empty folder: {path}")

    print("All files have been moved to the current directory!")

# download_kaggle("shaunthesheep/microsoft-catsvsdogs-dataset")

### Import libraries

In [ ]:
from sklearn.model_selection import train_test_split
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Show ERROR message only
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import pandas as pd
import torch
import psutil

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import CSVLogger
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
import cv2
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow import keras
from tensorflow.keras.models import Model
from sklearn.manifold import TSNE

### 1. Split the dataset 

In [ ]:
def clear_directory(directory):
    """Clear all files and subdirectories inside the given directory."""
    if os.path.exists(directory):
        for filename in os.listdir(directory):
            file_path = os.path.join(directory, filename)
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)

def dataset_split(data_dir, test_size=0.2, random_seed=42):
    # Get all subdirectories (categories) inside data_dir
    subdirs = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d)) and d not in ['train', 'test']]
    
    if len(subdirs) < 2:
        raise ValueError("At least two subdirectories (classes) are required in the dataset folder.")

    # Store file paths and corresponding labels
    files = []
    labels = []

    for idx, subdir in enumerate(subdirs):
        subdir_path = os.path.join(data_dir, subdir)
        subdir_files = [os.path.join(subdir_path, f) for f in os.listdir(subdir_path) if f.endswith('.jpg')]
        
        files.extend(subdir_files)
        labels.extend([idx] * len(subdir_files))  # Use index as the label

    # Split into training and test sets
    train_files, test_files, train_labels, test_labels = train_test_split(files, labels, test_size=test_size, random_state=random_seed, stratify=labels)

    # Define train/test directory paths
    train_dir = os.path.join(data_dir, 'train')
    test_dir = os.path.join(data_dir, 'test')

    # Clear the contents of train/ and test/ directories but keep the folders themselves
    clear_directory(train_dir)
    clear_directory(test_dir)

    # Recreate train and test directories if they are removed
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Create subdirectories for each class inside train and test directories
    for subdir in subdirs:
        os.makedirs(os.path.join(train_dir, subdir), exist_ok=True)
        os.makedirs(os.path.join(test_dir, subdir), exist_ok=True)

    # Move files to their respective folders
    for file, label in zip(train_files, train_labels):
        shutil.copy(file, os.path.join(train_dir, subdirs[label]))  # Map index to subdirectory name

    for file, label in zip(test_files, test_labels):
        shutil.copy(file, os.path.join(test_dir, subdirs[label]))

    print("The dataset has been successfully split into training and test sets!")
    print(f"Total training samples: {len(train_files)}")
    print(f"Total test samples: {len(test_files)}")
    

### 2. Data augmentation

In [ ]:
def resize_with_aspect_ratio(image, target_size=(150,150)):
    """
    Resize the image while maintaining the aspect ratio and fill blank areas to match the target size.
    """
    width, height = image.size
    aspect_ratio = width / height

    # Calculate new dimensions
    if aspect_ratio > 1:  # Width is greater than height
        new_width = target_size[0]
        new_height = int(target_size[0] / aspect_ratio)
    else:  # Height is greater than width
        new_height = target_size[1]
        new_width = int(target_size[1] * aspect_ratio)

    # Resize the image
    resized_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)

    # Fill blank areas
    new_image = Image.new("RGB", target_size, (255, 255, 255))  # White background
    new_image.paste(resized_image, ((target_size[0] - new_width) // 2, (target_size[1] - new_height) // 2))

    return new_image

def load_supported_extensions(file_path):
    with open(file_path, 'r') as f:
        # Read all extensions and remove newline characters and whitespace
        return [line.strip().lower() for line in f.readlines()]

def process_images_in_directory(data_dir, target_size=(150,150), supported_doc='supported_extensions.txt'):
    """
    Iterate through each image in the directory, resize it while maintaining the aspect ratio, 
    fill blank areas, save the modified image, and update file paths and labels.

    Parameters:
    - data_dir: Directory containing the dataset.
    - target_width: Target width.
    - target_height: Target height.
    """
    # Get all class names in the directory
    class_names = [d for d in os.listdir(os.path.join(data_dir, 'train')) if os.path.isdir(os.path.join(data_dir, 'train', d))]
    
    # Initialize lists to store file paths and labels
    train_files = []
    test_files = []
    train_labels = []
    test_labels = []

    # Load supported image formats
    image_extensions = load_supported_extensions(supported_doc)

    # Iterate through each class
    for class_name in class_names:
        # Get class directories for training and testing sets
        train_class_dir = os.path.join(data_dir, 'train', class_name)
        test_class_dir = os.path.join(data_dir, 'test', class_name)
        
        # Process images in both training and testing sets
        for class_dir, files_list, labels_list in [
            (train_class_dir, train_files, train_labels),
            (test_class_dir, test_files, test_labels)
        ]:

            for image_name in os.listdir(class_dir):
                image_path = os.path.join(class_dir, image_name)
                # Check if the file is an image
                if os.path.splitext(image_name)[1].lower() in image_extensions:
                    # Open the image
                    try:
                        with Image.open(image_path) as img:
                            # Resize and fill blank areas
                            new_img = resize_with_aspect_ratio(img, target_size)
                            
                            # Overwrite and save the modified image
                            new_img.save(image_path)  # Directly overwrite the original file
                            
                            # Update file path and label lists
                            files_list.append(image_path)
                            labels_list.append(class_name)
                            
                            # print(f"Processed and saved: {image_path}")
                            
                    except Exception as e:
                        print(f"Unable to process image {image_path}: {e}")
                        # Delete unprocessable images
                        os.remove(image_path)
                        print(f"Deleted unprocessable image {image_path}")
    
    # Output updated file paths and labels
    print(f"Number of training set file paths: {len(train_files)}")
    print(f"Number of testing set file paths: {len(test_files)}")

    # Save label information to a CSV file
    label_data = [{'image_path': file, 'label': label} for file, label in zip(train_files + test_files, train_labels + test_labels)]
    label_df = pd.DataFrame(label_data)
    label_df.to_csv(os.path.join(data_dir, 'labels.csv'), index=False)
    
    return train_files, test_files, train_labels, test_labels


In [ ]:
def calculate_batch_size(total_data, max_batch_size, image_size, validation_split=0.2, target_memory_usage=0.8):
    """
    Dynamically calculates the batch size, considering GPU and memory limitations.
    
    Parameters:
    - total_data: Total number of data points.
    - max_batch_size: Maximum batch size.
    - image_size: Size of each image (width, height, channels).
    - validation_split: Fraction of data to be used for validation.
    - target_memory_usage: Target memory usage percentage.

    Returns:
    - batch_size: Dynamically calculated batch size.
    """
    # Get GPU memory size
    if torch.cuda.is_available():
        print("GPU found.")
        gpu_memory = torch.cuda.get_device_properties(0).total_memory  # in bytes
    else:
        gpu_memory = 0  # Ignore if no GPU available

    # Get total system memory size
    system_memory = psutil.virtual_memory().total  # in bytes

    # Estimate memory usage per image in bytes
    image_memory = image_size[0] * image_size[1] * image_size[2] * 4  # Assuming each pixel is 4 bytes (float32)

    # Calculate max batch size based on GPU or memory
    if gpu_memory > 0:
        max_gpu_batch_size = int(gpu_memory * target_memory_usage // image_memory)
    else:
        max_gpu_batch_size = total_data  # If no GPU, use system memory to control batch size
    
    # Calculate max batch size based on system memory
    max_system_batch_size = int(system_memory * target_memory_usage // image_memory)

    # Consider both GPU and system memory limitations
    batch_size = min(max_batch_size, max_gpu_batch_size, max_system_batch_size, total_data // 10)

    # Check if there is enough data for the validation split
    if total_data * (1 - validation_split) < batch_size:
        raise ValueError(f"Not enough data to create a validation split. The total data size is too small for the batch size {batch_size}.")

    return batch_size

In [ ]:
def create_image_generators(data_dir, batch_size =64, validation_split=0.2, target_size=(150, 150)):
    """
    Creates image data generators for training and validation sets with data augmentation.
    Automatically detects the number of classes based on subdirectories in the 'train' directory.

    Parameters:
    - data_dir: Directory where the dataset is located. It should contain 'train' and optionally 'test' subdirectories.
    - target_size: The size of the images to which the input images are resized. Defaults to (150, 150).
    - batch_size: The number of samples per batch of data. Defaults to 64.
    - validation_split: Fraction of the dataset to be reserved for validation. Defaults to 0.2 (20% validation set, 80% training set).

    Returns:
    - train_generator: A generator that yields batches of augmented image data for training.
    - validation_generator: A generator that yields batches of image data for validation.

    Notes:
    - The function applies data augmentation techniques to the training data, including rescaling, rotation, shifting, shearing, zooming, and flipping.
    - The validation set is only rescaled without augmentation.
    - The function dynamically determines whether to use 'binary' or 'categorical' class mode based on the number of detected classes.
    - If no subdirectories (representing classes) are found in 'train', an error is raised.
    """
    
    # Data augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,         # Normalization
        rotation_range=20,      # Rotation
        width_shift_range=0.2,  # Horizontal shift
        height_shift_range=0.2, # Vertical shift
        shear_range=0.2,        # Shear transformation
        zoom_range=0.2,         # Zoom
        horizontal_flip=True,   # Horizontal flip
        validation_split=validation_split  # Train/validation split
    )

    # Data augmentation is not applied to the validation set
    val_datagen = ImageDataGenerator(rescale=1./255, validation_split=validation_split)  # Only rescaling for validation set

    # Data augmentation is not applied to the test set
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Dynamically detect the number of classes
    class_names = os.listdir(os.path.join(data_dir, 'train'))
   
    if not class_names:
        raise ValueError("No subdirectories found in the 'train' directory. Ensure that there are class subdirectories.")

    num_classes = len(class_names)

    # Set class_mode based on number of classes
    if num_classes == 2:
        class_mode = 'binary'
    else:
        class_mode = 'categorical'
    
    # print(class_mode)

    # Training set
    train_generator = train_datagen.flow_from_directory(
        os.path.join(data_dir, 'train'),  
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode,
        subset='training'
    )

    # Validation set
    validation_generator = val_datagen.flow_from_directory(
        os.path.join(data_dir, 'train'),
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode,
        subset='validation',  # Use the validation subset
        shuffle = False
    )

    # Test set
    test_generator = test_datagen.flow_from_directory(
        os.path.join(data_dir, 'test'),
        target_size=target_size,  # Use the same targe size as training set
        batch_size=batch_size,
        class_mode=None,  # No labels in test set
        shuffle=False
    )

    return train_generator, validation_generator, test_generator


### 3. CNN

In [ ]:
# Build CNN model
def build_model(
    input_shape=(150, 150, 3), 
    conv_layers=[(32, (3,3)), (64, (3,3)), (128, (3,3))], 
    pool_size=(2,2),
    dense_units=512,
    dropout_rate=0.5,
    output_units=1,
    output_activation='sigmoid',
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
):
    model = Sequential()
 
    # Add convolutional and pooling layers
    for i, (filters, kernel_size) in enumerate(conv_layers):
        if i == 0:
            model.add(Conv2D(filters, kernel_size, activation='relu', input_shape=input_shape))
        else:
            model.add(Conv2D(filters, kernel_size, activation='relu'))
        model.add(MaxPooling2D(pool_size=pool_size))
    
    model.add(Flatten())  # Flatten layer
    model.add(Dense(dense_units, activation='relu'))  # Fully connected layer
    model.add(Dropout(dropout_rate))  # Dropout layer
    model.add(Dense(output_units, activation=output_activation))  # Output layer
    
    # Compile the model
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    # Print model structure
    model.summary()
    
    return model

### 4. Train the model

In [12]:
def train_model(model_save_as, model, train_generator, validation_generator, csv_log, epochs = 10):
    # Create a CSVLogger callback to save the training history to a local CSV file
    csv_logger = CSVLogger(csv_log, append=True)
    
    # Train the model and use the CSVLogger callback
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        callbacks= [csv_logger]
    )

    # After training, the training history will be saved in 'training_history.csv'
    print("history has been save in f'{csv_log}'. ")

    # Save the model using Keras 
    model.save(f"{model_save_as}")

    print("model has been save in f'{model_save_as}'.")
    return history

### 5. Evaluate the model

In [ ]:
def evaluate_model(history):

    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.show()

### 6. Predict on the new data

In [ ]:
def predict_data():

    img_path = './PetImages/test/cat/32.jpg'
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    if prediction[0] > 0.5:
        print("Prediction: 🐶")
    else:
        print("Prediction: 🐱")


### Further exploration on model evaluation

In [ ]:
def plot_accuracy_loss(axes, history_df):
    """Plots accuracy and loss curves in the given axes."""
    if axes is None or len(axes)<2:
        return
    ax1, ax2 = axes
    # Accuracy
    ax1.plot(history_df["epoch"], history_df["accuracy"], label="Training Accuracy")
    ax1.plot(history_df["epoch"], history_df["val_accuracy"], label="Validation Accuracy")
    ax1.set_title("Accuracy Curve")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Accuracy")
    ax1.legend()

    # Loss
    ax2.plot(history_df["epoch"], history_df["loss"], label="Training Loss")
    ax2.plot(history_df["epoch"], history_df["val_loss"], label="Validation Loss")
    ax2.set_title("Loss Curve")
    ax2.set_xlabel("Epochs")
    ax2.set_ylabel("Loss")
    ax2.legend()

def plot_confusion_matrix(ax, y_true, y_pred, class_names):
    
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, ax=ax)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title('Confusion Matrix')

def plot_roc_curve(ax, y_true, y_scores):
  
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    ax.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {roc_auc:.2f})')
    ax.plot([0, 1], [0, 1], color='gray', linestyle='--')
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('ROC Curve')
    ax.legend()

def get_img_array(img_path, size):
    # `img` is a PIL image of size height * width
    img = keras.utils.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (height, width, 3)
    array = keras.utils.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, height, width, 3)
    array = np.expand_dims(array, axis=0)
    array = array / 255.0  # Normalize pixel values to the range [0, 1] for neural network processing
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names):
    """
    Generate a Grad-CAM heatmap for a given image.
    
    Args:
        img_array (numpy array): Preprocessed input image.
        model (tf.keras.Model): Trained model.
        last_conv_layer_name (str): Name of the last convolutional layer.
        classifier_layer_names (list of str): List of classifier layer names after the convolutional layer.
    
    Returns:
        numpy array: Normalized heatmap.
    """
    # Extract the last convolutional layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Define a classifier model from the output of the last convolutional layer
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    # Compute gradients of the predicted class score with respect to feature maps
    with tf.GradientTape() as Tape:
        last_conv_layer_output = last_conv_layer_model(img_array)
        Tape.watch(last_conv_layer_output)
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    grads = Tape.gradient(top_class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2)).numpy()

    # Weight the feature map channels by their importance
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(last_conv_layer_output, axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)  # Normalize heatmap
    return heatmap

def plot_grad_cam(axes, misclassified_dir, target_size, model, last_conv_layer_name, classifier_layer_names, alpha):
    """
    Plot Grad-CAM heatmap and overlay images on given axes.
    
    Args:
        axes (list): List of matplotlib axes.
        misclassified_dir (str): Directory containing misclassified images.
        target_size (tuple): Target size for image preprocessing.
        model (tf.keras.Model): Trained model.
        last_conv_layer_name (str): Name of the last convolutional layer.
        classifier_layer_names (list of str): List of classifier layer names.
        alpha (float): Weighting factor for heatmap overlay.
    """
    if axes is None or len(axes) < 3:
        return
    
    img_paths = [os.path.join(subdir, file) for subdir, _, files in os.walk(misclassified_dir) for file in files]
    n = len(img_paths)
    if len(axes) < n * 3:
        print(f"Not enough axes. Required at least {n * 3}.")
        return
    
    for i, img_path in enumerate(img_paths):
        ax0, ax1, ax2 = axes[i * 3], axes[i * 3 + 1], axes[i * 3 + 2]
        img_array = get_img_array(img_path, target_size)
        heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names)

        img = keras.utils.load_img(img_path)
        img = keras.utils.img_to_array(img)
        ax0.imshow(img.astype(np.uint8))
        ax0.axis('off')
        ax0.set_title('Original Image')

        heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
        heatmap = np.uint8(255 * heatmap)
        heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
        
        superimposed_img = heatmap * alpha + img * (1 - alpha)
        superimposed_img = np.clip(superimposed_img, 0, 255).astype(np.uint8)

        ax1.imshow(heatmap, cmap='viridis')
        ax1.axis('off')
        ax1.set_title('Grad-CAM Heatmap')

        ax2.imshow(superimposed_img)
        ax2.axis('off')
        ax2.set_title('Grad-CAM Overlay')

def plot_tsne(ax, model, test_generator, class_names,  n_components, random_state, perplexity, n_iter):
    """
    Plots t-SNE visualization of feature embeddings from the CNN model.
    
    Args:
        ax (matplotlib axis): Axis to plot the t-SNE visualization.
        model (tf.keras.Model): Trained CNN model.
        test_generator (ImageDataGenerator): Test data generator.
        class_names (list): List of class names (e.g., ['cat', 'dog']).
    """
    # Extract features from the model (before the final classification layer)
    feature_extractor = tf.keras.Model(inputs=model.inputs, outputs=model.layers[-2].output)
    features = feature_extractor.predict(test_generator)
    labels = test_generator.classes

    # Apply t-SNE to reduce features to 2D
    tsne = TSNE(n_components=n_components, random_state=random_state, perplexity=perplexity, n_iter=n_iter)
    features_2d = tsne.fit_transform(features)

    # Plot t-SNE results
    scatter = ax.scatter(features_2d[:, 0], features_2d[:, 1], c=labels, cmap='viridis', s=20)
    ax.set_title('t-SNE Visualization of Feature Embeddings')
    ax.set_xlabel('t-SNE Dimension 1')
    ax.set_ylabel('t-SNE Dimension 2')

    # plt.colorbar(scatter, ax=ax, ticks=range(len(class_names)), label='Class')
    # Create a colorbar with class names
    cbar = plt.colorbar(scatter, ax=ax, ticks=range(len(class_names)))
    cbar.set_ticklabels([f'{class_names[i]} ({i})' for i in range(len(class_names))])  # Add both class names and labels
    cbar.set_label('Class')

    ax.set_xticks([])
    ax.set_yticks([])

def evaluation_visualizations(log_csv, model_save_as, test_generator, data_dir, target_size, misclassified_dir, max_per_class, n_components, random_state, perplexity, n_iter):
    """Dynamically generates a grid and fills it with various visualization functions."""
    # Load the training history from CSV
    history_df = pd.read_csv(log_csv)

    # Load the trained model
    model = load_model(model_save_as) 

    num_classes = len(test_generator.class_indices)  # Get the number of classes
    # Retrieve class names
    class_names = list(test_generator.class_indices.keys())
    print("Class names:", class_names)

    # Get predictions
    y_pred = (model.predict(test_generator) > 0.5).astype(int)
    y_scores = model.predict(test_generator)  # Get probabilities for class 1

    if num_classes > 2:
        # For multi-class classification, convert to class indices
        y_pred = np.argmax(y_pred, axis=1)
    
    y_true = test_generator.classes  # Get true labels

    # **Save misclassified samples**
    # Clear the directory
    clear_directory(misclassified_dir)
    os.makedirs(misclassified_dir, exist_ok=True)

    y_pred = y_pred.ravel()
    y_true = y_true.ravel()

    misclassified_indices = np.where(y_true != y_pred)[0]  # Indices of misclassified samples
    misclassified_counts = {}  # Track misclassification count per class pair
    total_misclassified_images = 0  # Variable to track the total number of misclassified images

    for idx in misclassified_indices:
        true_label = int(y_true[idx])
        pred_label = int(y_pred[idx])

        # Track the misclassification count for (true_label, pred_label) pair
        misclass_key = (true_label, pred_label)
        if misclassified_counts.get(misclass_key, 0) >= max_per_class:
            continue

        # Get image path
        img_path = os.path.join(data_dir, test_generator.filenames[idx])  
        img = Image.open(img_path)

        # Ensure class directory exists
        class_dir = os.path.join(misclassified_dir, f"class_{class_names[true_label]}_pred_{class_names[pred_label]}")
        os.makedirs(class_dir, exist_ok=True)

        # Save the misclassified image
        save_path = os.path.join(class_dir, f"{misclassified_counts.get(misclass_key, 0)}.png")
        img.save(save_path)

        # Update misclassification count
        misclassified_counts[misclass_key] = misclassified_counts.get(misclass_key, 0) + 1
        total_misclassified_images += 1 
    print(f"Saved {total_misclassified_images} misclassified images in '{misclassified_dir}'")

    # Find the last Conv2D layer
    last_conv_layer_name = None
    for layer in reversed(model.layers):  # Iterate from the last layer
        if isinstance(layer, tf.keras.layers.Conv2D):
            last_conv_layer_name = layer.name
            break

    print("Last convolutional layer:", last_conv_layer_name)

    # Get classifier layer names (layers after the last Conv2D layer)
    classifier_layer_names = []
    found_conv_layer = False

    for layer in model.layers:
        if layer.name == last_conv_layer_name:
            found_conv_layer = True  # Mark when the last Conv2D layer is found
        if found_conv_layer:
            classifier_layer_names.append(layer.name)

    # Remove the first element (which is the last Conv2D layer itself)
    classifier_layer_names.pop(0)

    print("Classifier layers:", classifier_layer_names)

    alpha = 0.4
    # List of visualization functions with parameters
    viz_functions = [
        {"func": plot_accuracy_loss, "params": {"history_df": history_df}, "ax_count": 2},
        {"func": plot_confusion_matrix, "params": {"y_true": y_true, "y_pred": y_pred, "class_names": class_names}, "ax_count": 1},
        {"func": plot_roc_curve, "params": {"y_true": y_true, "y_scores": y_scores}, "ax_count": 1},
        {"func": plot_grad_cam, 
            "params": {
                "misclassified_dir": misclassified_dir, 
                "target_size": target_size, 
                "model": model,
                "last_conv_layer_name": last_conv_layer_name,
                "classifier_layer_names": classifier_layer_names,
                "alpha": alpha
            }, 
            "ax_count": total_misclassified_images * 3
        },
        {"func": plot_tsne, 
            "params": {
                "model": model, 
                "test_generator": test_generator,
                "class_names": class_names,
                "n_components": n_components, 
                "random_state": random_state, 
                "perplexity": perplexity, 
                "n_iter": n_iter
                }, 
            "ax_count": 1
        },
    ]
    
    # Dynamically determine grid size
    total_rows = sum(v["ax_count"] // 3 if v["func"] == plot_grad_cam else 1 for v in viz_functions)
    fig, axes = plt.subplots(nrows=total_rows, ncols=3, figsize=(18, 5 * total_rows))    
    
    axes = np.array(axes).flatten()  # Flatten axes array for easy indexing

    # Execute visualization functions
    i = 0
    for viz_info in viz_functions:
        func = viz_info["func"]
        params = viz_info["params"]
        ax_count = viz_info["ax_count"]
        
        if ax_count == 1:
            func(axes[i], **params)  # Use a single axis
        else:
            func(axes[i:i+ax_count], **params)  # Pass multiple axes
        i += ax_count  # Increment index

    # Remove unused subplots
    for j in range(i, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

In [ ]:
def main():
    data_folder      = 'PetImages'
    data_dir         = './' + data_folder
    model_type       = 'cnn'
    model_save_as    = data_folder + model_type + '.keras'
    test_size        = 0.2
    random_seed      = 42
    target_size      = (150,150)
    supported_doc    = 'supported_extensions.txt'
    csv_log          = 'training_history.csv'
    misclassified_dir= os.path.join(data_dir, 'misclassified_images')  # Store misclassified images in data_dir
    max_per_class    = 10 # Maximum of 10 misclassified images per category
    n_components     = 2 # Set the number of dimensions for the output (2D for visualization)
    random_state     = 42 # Set the seed for reproducibility of results
    perplexity       = 30 # Control the balance between local and global data relationships
    n_iter           = 1000 # Set the number of iterations for the optimization process


    # 0. Download the dataset
    download_kaggle("shaunthesheep/microsoft-catsvsdogs-dataset")

    # 1. Split the dataset
    dataset_split(data_dir, test_size, random_seed)

    # 2. Data Augmentation
    # 2.1 Resize the images
    train_files, test_files, train_labels, test_labels = process_images_in_directory(data_dir, target_size, supported_doc)

    # 2.2 Calculate the batch size
    total_data = len(train_files)
    image_size = (150, 150, 3)  # Assuming each image has dimensions 150x150 with 3 channels
    validation_split = 0.2
    target_memory_usage = 0.8
    batch_size = calculate_batch_size(total_data, max_batch_size, image_size, validation_split, target_memory_usage)
    # print("batch_size: ", batch_size)
    
    # 2.3 Create Image Generators
    train_generator, validation_generator, test_generator = create_image_generators(data_dir, batch_size, validation_split, target_size)
    
    # 3. Build a CNN model
    num_classes = len(train_generator.class_indices)  # Get the number of categories
    if num_classes == 2:
        output_units = 1
        output_activation = 'sigmoid'
        loss = 'binary_crossentropy'
    else:
        output_units = num_classes
        output_activation = 'softmax'
        loss = 'categorical_crossentropy'

    input_shape = (150, 150, 3)
    conv_layers = [(32, (3,3)), (64, (3,3)), (128, (3,3)), (256, (3,3))]
    pool_size = (2,2)
    dense_units = 1024
    dropout_rate = 0.4
    optimizer = 'adam'
    metrics = ['accuracy']

    cnn_model = build_model(
        input_shape, 
        conv_layers, 
        pool_size,
        dense_units,
        dropout_rate,
        output_units,
        output_activation,
        optimizer,
        loss,
        metrics)
    
    # 4. Train the model
    epochs = 10
    history = train_model(model_save_as, cnn_model, train_generator, validation_generator, csv_log, epochs)
    
    # 5. Evaluate the model
    evaluate_model(history)
    
    # 6. Predict on new data
    # Load the model
    model = load_model(model_save_as)  # Replace with your model path
    
    # Load and preprocess an image
    img_path = './PetImages/test/cat/32.jpg'
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    if prediction[0] > 0.5:
        print("Prediction: Dog 🐶")
    else:
        print("Prediction: Cat 🐱")


    # Further exploration on model evaluation
    evaluation_visualizations(csv_log, model_save_as, test_generator, data_dir, target_size, misclassified_dir, max_per_class, n_components, random_state, perplexity, n_iter)

if __name__ == "__main__":
    main()
